<a href="https://colab.research.google.com/github/nightowldamon/hello-AI/blob/chatbot/xAI_chatbot_UI_LangChain_Gradio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install gradio langchain-community langchain langchain-xai langchain-anthropic langchain-openai

In [4]:
import os
from google.colab import userdata

os.environ["XAI_API_KEY"] = userdata.get('XAI_API_KEY')
os.environ["ANTHROPIC_API_KEY"] = userdata.get('ANTHROPIC_API_KEY')
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')


In [5]:
from langchain_xai import ChatXAI
from langchain_anthropic import ChatAnthropic
from langchain_openai import ChatOpenAI
from langchain.schema import AIMessage, HumanMessage, SystemMessage
import gradio as gr

In [6]:
system_message = "You are Grok, an AI with a sense of humor inspired by the Hitchhiker's Guide to the Galaxy. Your responses should be witty, slightly rebellious, and always strive to be maximally helpful."

In [ ]:

# Initialize xAI chat model
llm = ChatXAI(temperature=0.5, model="grok-beta", streaming=True)

# Initialize OpenAI chat model
llm = ChatOpenAI(temperature=0.2, model='gpt-4o', streaming=True)

# Initialize Anthropic chat model
# llm = ChatAnthropic(model='claude-3-5-sonnet-20241022', streaming=True)



def stream_response(message, history):
    print(f"Input: {message}. History: {history}\n")

    history_langchain_format = []
    history_langchain_format.append(SystemMessage(content=system_message))

    for human, ai in history:
        history_langchain_format.append(HumanMessage(content=human))
        history_langchain_format.append(AIMessage(content=ai))

    if message is not None:
        history_langchain_format.append(HumanMessage(content=message))
        partial_message = ""
        for response in llm.stream(history_langchain_format):
            partial_message += response.content
            yield partial_message


demo_interface = gr.ChatInterface(
    stream_response,
    textbox=gr.Textbox(placeholder="Send to the LLM...",
                       container=False,
                       autoscroll=True,
                       scale=7),
)

demo_interface.launch(share=True, debug=True)